In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import random
import pickle
from scipy.stats import gaussian_kde
# from sklearn.preprocessing import StandardScaler
import train
import sort_data
import matplotlib.pyplot as plt
import shutil
import scipy.io as scio

In [2]:
class StandardScaler():
    def __init__(self):
        self.var = None
        self.mean = None
    def fit(self, x):
        self.mean = x.mean(axis=0)
        self.var = x.var(axis=0)
        return self
    def transform(self,x):
        return (x-self.mean)/np.sqrt(self.var)
    
    def inverse_transform(self,x):
        return (x*np.sqrt(self.var))+self.mean


In [3]:
def plan(chose_input,save_path,band_norm=False):
    sort_data.get_data(chose_input)
    
    for file in sorted(os.listdir("../0 save data/train data/")):
        
        name = file.split(".pkl")[0]
#         if name not in  ["tchl_a"]: continue
        with open("../0 save data/train data/"+name+".pkl",'rb') as f:
            x,y,raw_x,raw_y,case1 = pickle.load(f)
        with open("../0 save data/train data 7 day/"+name+".pkl",'rb') as f:
            px,py,raw_px,raw_py,case1_7day = pickle.load(f)
        if band_norm:
            Rrs_norm = x[:,-10:]
            Rrs_norm = (Rrs_norm-np.mean(Rrs_norm,axis=1)[:,np.newaxis])/np.std(Rrs_norm,axis=1)[:,np.newaxis]
            x[:,-10:] = Rrs_norm
            Rrs_norm = px[:,-10:]
            Rrs_norm = (Rrs_norm-np.mean(Rrs_norm,axis=1)[:,np.newaxis])/np.std(Rrs_norm,axis=1)[:,np.newaxis]
            px[:,-10:] = Rrs_norm
            
            
        error_sl,error_lg,error_tr = {},{},{}
        this_RMSE = 9999

        ### standard, get train_x, train_y
        idx = random.sample(range(y.shape[0]),y.shape[0])
        x,y = x[idx], y[idx]
        raw_x, raw_y = {item:raw_x[item][idx] for item in raw_x}, raw_y[idx]
        case1 = case1[idx]

        scaler_x = StandardScaler().fit(x)
        scaler_y = StandardScaler().fit(y)

        x = scaler_x.transform(x.copy())
        y = scaler_y.transform(y.copy())
        px = scaler_x.transform(px.copy())
        py = scaler_y.transform(py.copy())


        ### train and test
        loss,loss_cv,predict_cv = train.training_main(px.copy(),py.copy(),x.copy(),y.copy())
        shutil.copy("../0 save data/best-trained-model.pth","../0 save data/pigments-model/"+name.replace("-","_")+"-model.pth")
        save_param = {
            "name": name,
            "pretrain_inputs": raw_px,
            "pretrain_outputs": raw_py,
            "cv_inputs": raw_x,
            "cv_outputs": raw_y,
            "scaler_x": scaler_x,
            "scaler_y": scaler_y,
            "train_loss": loss,
            "cv_loss": loss_cv,
            "target_cv": scaler_y.inverse_transform(y.copy()),
            "predict_cv": scaler_y.inverse_transform(predict_cv.copy()),
            "case1": case1
        }
        
        print(name,"done!")
#         print(name,)
        with open(save_path+name.replace("-","_")+".pkl",'wb') as f:
            pickle.dump(save_param, f)
        scio.savemat(save_path+name.replace("-","_")+".mat", save_param)


In [4]:
chose_input = {"sst":True,
               "par":True,
               "Zeu":False,
               "Rrs":True,
               "chl_a":False,
               "Rrs_band":False,
               "bbp_aph":False,
               "lat_lon":True}
path = "../0 save data/NN result (only case1) (DL-PPCE-1) (band norm test3)/"
if not os.path.exists(path):
    os.mkdir(path)
# sort_data.get_data(chose_input)

plan(chose_input,path,band_norm=True)

fuco 1103 (1103, 14) (1103, 1)
perid 1088 (1088, 14) (1088, 1)
hex-fuco 1104 (1104, 14) (1104, 1)
but-fuco 1084 (1084, 14) (1084, 1)
allo 1023 (1023, 14) (1023, 1)
tchl_b 838 (838, 14) (838, 1)
zea 1100 (1100, 14) (1100, 1)
tchl_a 1111 (1111, 14) (1111, 1)
diato 876 (876, 14) (876, 1)
diadino 942 (942, 14) (942, 1)
mv_chl_a 501 (501, 14) (501, 1)
dv_chl_a 778 (778, 14) (778, 1)
chl_c12 901 (901, 14) (901, 1)
chl_c3 922 (922, 14) (922, 1)
neo 779 (779, 14) (779, 1)
viola 880 (880, 14) (880, 1)
lut 742 (742, 14) (742, 1)
pras 669 (669, 14) (669, 1)
chl_a 860 (860, 14) (860, 1)
chlide_a 841 (841, 14) (841, 1)
fuco 1330 (1330, 14) (1330, 1)
perid 1311 (1311, 14) (1311, 1)
hex-fuco 1325 (1325, 14) (1325, 1)
but-fuco 1305 (1305, 14) (1305, 1)
allo 1212 (1212, 14) (1212, 1)
tchl_b 1039 (1039, 14) (1039, 1)
zea 1311 (1311, 14) (1311, 1)
tchl_a 1334 (1334, 14) (1334, 1)
diato 1001 (1001, 14) (1001, 1)
diadino 1093 (1093, 14) (1093, 1)
mv_chl_a 581 (581, 14) (581, 1)
dv_chl_a 921 (921, 14) (921,

In [5]:
# chose_input = {"sst":True,
#                "par":True,
#                "Zeu":True,
#                "Rrs":True,
#                "chl_a":False,
#                "Rrs_band":True,
#                "bbp_aph":False,
#                "lat_lon":True}
# path = "../0 save data/NN result (only case1) (DL-PPCE-2)/"
# if not os.path.exists(path):
#     os.mkdir(path)
# # sort_data.get_data(chose_input)

# plan(chose_input,path)

In [6]:
# chose_input = {"sst":True,
#                "par":True,
#                "Zeu":True,
#                "Rrs":False,
#                "chl_a":False,
#                "Rrs_band":False,
#                "bbp_aph":True,
#                "lat_lon":True}
# path = "../0 save data/NN result (only case1) (DL-PPCE-3)/"
# if not os.path.exists(path):
#     os.mkdir(path)
# # sort_data.get_data(chose_input)

# plan(chose_input,path)

In [7]:
# chose_input = {"sst":True,
#                "par":True,
#                "Zeu":True,
#                "Rrs":False,
#                "chl_a":True,
#                "Rrs_band":False,
#                "bbp_aph":False,
#                "lat_lon":True}
# path = "../0 save data/NN result (only case1) (DL-PPCE-4)/"
# if not os.path.exists(path):
#     os.mkdir(path)
# # sort_data.get_data(chose_input)

# plan(chose_input,path)

In [8]:
# chose_input = {"sst":True,
#                "par":True,
#                "Zeu":False,
#                "Rrs":True,
#                "chl_a":False,
#                "Rrs_band":False,
#                "bbp_aph":False,
#                "lat_lon":True}
# path = "../0 save data/NN result (only case1) (DL-PPCE-5)/"
# if not os.path.exists(path):
#     os.mkdir(path)
# # sort_data.get_data(chose_input)

# plan(chose_input,path)

In [9]:
# chose_input = {"sst":True,
#                "par":False,
#                "Zeu":True,
#                "Rrs":True,
#                "chl_a":False,
#                "Rrs_band":False,
#                "bbp_aph":False,
#                "lat_lon":True}
# path = "../0 save data/NN result (only case1) (DL-PPCE-6)/"
# if not os.path.exists(path):
#     os.mkdir(path)
# # sort_data.get_data(chose_input)

# plan(chose_input,path)

In [10]:
# chose_input = {"sst":False,
#                "par":True,
#                "Zeu":True,
#                "Rrs":True,
#                "chl_a":False,
#                "Rrs_band":False,
#                "bbp_aph":False,
#                "lat_lon":True}
# path = "../0 save data/NN result (only case1) (DL-PPCE-7)/"
# if not os.path.exists(path):
#     os.mkdir(path)
# # sort_data.get_data(chose_input)

# plan(chose_input,path)

In [11]:
# chose_input = {"sst":True,
#                "par":False,
#                "Zeu":False,
#                "Rrs":True,
#                "chl_a":False,
#                "Rrs_band":False,
#                "bbp_aph":False,
#                "lat_lon":True}
# path = "../0 save data/NN result (only case1) (DL-PPCE-8)/"
# if not os.path.exists(path):
#     os.mkdir(path)
# # sort_data.get_data(chose_input)

# plan(chose_input,path)

In [12]:
# chose_input = {"sst":False,
#                "par":False,
#                "Zeu":False,
#                "Rrs":True,
#                "chl_a":False,
#                "Rrs_band":False,
#                "bbp_aph":False,
#                "lat_lon":True}
# path = "../0 save data/NN result (only case1) (DL-PPCE-9)/"
# if not os.path.exists(path):
#     os.mkdir(path)
# # sort_data.get_data(chose_input)

# plan(chose_input,path)

In [13]:
# chose_input = {"sst":False,
#                "par":False,
#                "Zeu":False,
#                "Rrs":True,
#                "chl_a":False,
#                "Rrs_band":False,
#                "bbp_aph":False,
#                "lat_lon":False}
# path = "../0 save data/NN result (only case1) (DL-PPCE-0)/"
# if not os.path.exists(path):
#     os.mkdir(path)
# # sort_data.get_data(chose_input)

# plan(chose_input,path)